# How to Flatten a Python `dict`

We've all had that moment.
We are trying to analyze a nested `dict`.
We start by peeling away the layers one at a time,
hoping to find the values.
Sometimes this approach works...
and sometimes you need to **flatten it!**.

## What do I mean by *flattening*?

There are many ways to *flatten* a `dict`, but what I
want is this:

* I want to start with an arbitrarily nested `dict`
  * Or `list`!
* I want all of the *values*
* I also want all of the *paths* to those values
* I want the *paths* to be valid Python code

At this point an example might help.

Imagine a classroom with students. If I wanted to store the *roster*
in a `dict` it could look like this:

In [ ]:
roster = {
    "students":[
        {
            "age": 25,
            "name": "John",
        },
        {
            "age": 30,
            "name": "Jane",
        }
    ],
    "class":{
        "title": "Philosophy 101",
        "id": 12345,
    },
}

Here are the *paths* and *values* I want, stored in a `pandas.DataFrame`
because `pandas` rocks!

In [ ]:
import pandas as pd

In [ ]:
roster_flattened = pd.DataFrame(
    {
        "path":[
            'roster["students"][0]["age"]',
            'roster["students"][0]["name"]',
            'roster["students"][1]["age"]',
            'roster["students"][1]["name"]',
            'roster["class"]["title"]',
            'roster["class"]["id"]',
        ],
        "value":[
            25,
            "John",
            30,
            "Jane",
            "Philosophy 101",
            12345,
        ],
    }
)
roster_flattened

## Let's Flatten Something!

Let's create our function that does the actual flattening.
I'll define it here, and explain parts of it later.

In [ ]:
from typing import Dict, List, Union, Tuple, Any

Flattenable = Union[Dict, List]
Path = str
Paths = List[Path]
Value = Any
Values = List[Value]

def flatten(obj:Flattenable, name:str)->Tuple[Paths, Values]:
    
    paths = []
    values = []
    
    def do_flattening(
        obj: Flattenable,
        path:Path,
    ):
        obj_type = type(obj)
        
        if dict == obj_type:
            for key, value in obj.items():
                new_path = f'{path}["{key}"]'
                new_obj = value
                do_flattening(new_obj, new_path)
    
        if list == obj_type:
            for i, item in enumerate(obj):
                new_path = f"{path}[i]"
                new_obj = item
                do_flattening(new_obj, new_path)
                
        else:
            paths.append(path)
            values.append(obj)
            
    do_flattening(obj, name)
    
    return paths, values


In [ ]:
paths, values = flatten(roster, "roster")

